<a href="https://colab.research.google.com/github/ekolonsky/RIA_news/blob/main/RIA_Novosti_EDA_n_grams_freq_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Frequency analysis with n-grams

In [2]:
# install packages for NLP
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import requests, re
from collections import Counter

In [3]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer(lang='ru')
# пример - морфологический разбора слова "идём" 
print(morph.parse('идём'))
normal = {}

[Parse(word='идём', tag=OpencorporaTag('VERB,impf,intr plur,1per,pres,indc'), normal_form='идти', score=0.5, methods_stack=((DictionaryAnalyzer(), 'идём', 1696, 2),)), Parse(word='идём', tag=OpencorporaTag('VERB,impf,intr sing,impr,incl'), normal_form='идти', score=0.5, methods_stack=((DictionaryAnalyzer(), 'идём', 1696, 11),))]


In [5]:
  # Функция для разбиений на токены

separators = ";", ",", '"', ':', '.','!', '?', ' ', '`', '%','$','*','(',')'
regex_punctuation = '|'.join(map(re.escape, separators))
url = 'https://raw.githubusercontent.com/ekolonsky/RIA_news/main/'


def get_stopwords():
    req = requests.get(url + 'stopwords.txt')
    ans = req.text.split()
    return ans
stopwords = get_stopwords() # затем сделаем загрузку из словаря стоп-слов
print(stopwords)

def get_whitelist():
    req = requests.get(url + 'whitelist.txt')
    ans = req.text.split()
    return ans
whitelist = get_whitelist() # затем сделаем загрузку из словаря стоп-слов
print(whitelist)



def normalize(word):
    word = word.lower()
    if word not in normal:
      normal[word] = morph.parse(word)[0].normal_form
    return normal[word]

def tokenize(text):
    text = text.lower()
    tokens = [normalize(word) for word in re.split(regex_punctuation,text)  
      if not word.isnumeric()
      and word != ''] 
    return [w for w in tokens if w not in stopwords] 

def generate_N_grams(tokens,ngram=1):
    temp=zip(*[tokens[i:] for i in range(0,ngram)])
    ans=[' '.join(ngram) for ngram in temp]
    return ans

# пример
generate_N_grams(tokenize('В Москве назвали победителей Гран-при 15 Московского кинофестиваля "Минотавр".'),3)

['а', 'ан', 'без', 'безусловно', 'благодаря', 'близ', 'будто', 'в', 'вблизи', 'ввиду', 'вглубь', 'вдоль', 'велено', 'верно', 'вероятно', 'включая', 'вместо', 'вне', 'внутри', 'возле', 'возможно', 'вокруг', 'вопреки', 'вправе', 'вслед', 'вследствие', 'вы', 'для', 'до', 'если', 'жаль', 'за', 'зато', 'значит', 'и', 'из', 'известно', 'из-за', 'из-по', 'из-под', 'или', 'исключая', 'к', 'как', 'когда', 'кое-кто', 'кое-что', 'коль', 'кроме', 'кто', 'кто-либо', 'кто-нибудь', 'кто-то', 'куда', 'ли', 'либо', 'между', 'многие', 'многое', 'можно', 'мы', 'на', 'над', 'надо', 'наконец-то', 'например', 'напротив', 'насчёт', 'негде', 'незачем', 'некого', 'некто', 'некуда', 'нельзя', 'немногие', 'немногое', 'необходимо', 'несмотря', 'нет', 'нету', 'нечего', 'нечто', 'ниже', 'никто', 'ничто', 'но', 'нужно', 'о', 'однако', 'около', 'он', 'она', 'они', 'оно', 'оное', 'от', 'относительно', 'перед', 'по', 'по-видимому', 'под', 'помимо', 'поскольку', 'после', 'посредством', 'пред', 'прежде', 'при', 'причём',

['москва назвать победитель',
 'назвать победитель гран-при',
 'победитель гран-при московский',
 'гран-при московский кинофестиваль',
 'московский кинофестиваль минотавр']

In [6]:
# функция для извлечения даты новости из строки со ссылкой на новость
def get_date(link):
  pattern = r'\/20[0-2][0-9][0-1][0-9][0-3][0-9]\/'
  result = re.search(pattern, link)

  if result:
    return result[0][1:-1]
  else:
    return ''
  return 

# пример
get_date('https://ria.ru/20041229/774359.html	Жерар Депардье открыл в Париже второй ресторан')

'20041229'

## Read and count n-grams

In [7]:
#%%time
url = 'https://raw.githubusercontent.com/ekolonsky/RIA_news/main/'
filenames = ['ria-{:02d}-1.txt'.format(i) for i in range(2,23)] 
filenames += ['ria-{:02d}-2.txt'.format(i) for i in range(2,22)]
#filenames = ['ria-{:02d}-1.txt'.format(i) for i in range(2,3)]  # quick test

ngrams_cnt = Counter()

YEAR = '2014'

for filename in filenames:
  print(filename)
  req = requests.get(url + filename)
  for line in req.text.splitlines():
    sep = line.find('\t')
    if sep == -1:
      continue
    link, news = line[:sep],line[sep+1:]
    tokens = tokenize(news)
    
    in_whitelist =  any(word in whitelist for word in tokens)  # filter news by words in whitelist

    date = get_date(link)
    year = date[:4]
    for n in [2, 3, 4]:
      ngrams = generate_N_grams(tokens,n)
      for ngram in ngrams:
        ngrams_cnt[(ngram, n, year < YEAR, in_whitelist)] +=1



ria-02-1.txt
ria-03-1.txt
ria-04-1.txt
ria-05-1.txt
ria-06-1.txt
ria-07-1.txt
ria-08-1.txt
ria-09-1.txt
ria-10-1.txt
ria-11-1.txt
ria-12-1.txt
ria-13-1.txt
ria-14-1.txt
ria-15-1.txt
ria-16-1.txt
ria-17-1.txt
ria-18-1.txt
ria-19-1.txt
ria-20-1.txt
ria-21-1.txt
ria-22-1.txt
ria-02-2.txt
ria-03-2.txt
ria-04-2.txt
ria-05-2.txt
ria-06-2.txt
ria-07-2.txt
ria-08-2.txt
ria-09-2.txt
ria-10-2.txt
ria-11-2.txt
ria-12-2.txt
ria-13-2.txt
ria-14-2.txt
ria-15-2.txt
ria-16-2.txt
ria-17-2.txt
ria-18-2.txt
ria-19-2.txt
ria-20-2.txt
ria-21-2.txt


In [9]:
len(ngrams_cnt), ngrams_cnt.most_common(20)

(27567934,
 [(('человек погибнуть', 2, True, False), 10614),
  (('не быть', 2, True, False), 7495),
  (('глава мид', 2, True, False), 6632),
  (('федеральный округ', 2, True, False), 6486),
  (('млн руб', 2, True, False), 6237),
  (('человек погибнуть', 2, False, False), 6008),
  (('принять участие', 2, True, False), 5957),
  (('глава мид', 2, False, False), 5769),
  (('теннисный турнир', 2, True, False), 5741),
  (('сборная россия', 2, True, False), 5633),
  (('мид рф', 2, True, False), 5310),
  (('не быть', 2, False, False), 5295),
  (('более тысяча', 2, False, False), 5002),
  (('планировать освещать', 2, True, True), 4741),
  (('событие который', 2, True, True), 4552),
  (('владимир путин', 2, True, False), 4547),
  (('мочь быть', 2, True, False), 4368),
  (('риа новость', 2, True, False), 4285),
  (('тысяча человек', 2, False, False), 4265),
  (('млрд руб', 2, True, False), 4153)])

## Save

In [17]:
import codecs

with open('counter-100.txt','w') as file:
    for key, value in ngrams_cnt.items():
        if value > 100:
          ngram = key[0]
          n = key[1]
          year = key[2]
          wl = key[3]
          file.write('{0};{1};{2};{3};{4}\n'.format(ngram, n, year, wl, value))